Imports

In [15]:
import sqlite3
import pandas as pd
from sqlite3 import Connection, Cursor
from pandas import DataFrame
import numpy as np

Function to write a data frame into a SQL table of an existing SQL database

In [20]:
def df_to_sql(df: DataFrame, cur: Cursor, table_name: str, db_name: str, conn: Connection):
    res = cur.execute("SELECT name FROM sqlite_master")
    if res.fetchone() is None:
        cur.execute(f"CREATE TABLE {table_name}(country, metro, state, population, lat, lng)")
    else:
        print('Table already exists.')
        cur.execute(f"DELETE FROM {table_name}")
    
    for index, row in df.iterrows():
        metro_str: str = str(row['Metro Area'])
        
        metro_index: int = metro_str.rfind("'")
        if metro_str.rfind("'") != -1:
            metro_str = metro_str[:metro_index] + "'" + metro_str[metro_index:] 
            
        pop_str: str = np.char.replace(row['Population'], ',', '')
        cur.execute(f"INSERT INTO {table_name} VALUES ('{str(row['Country'])}', '{metro_str}', '{str(row['State'])}', {int(pop_str)}, {float(row['lat'])}, {float(row['lng'])})")
    
    conn.commit()

Create a SQL database from metros.csv

In [26]:
df: DataFrame = pd.read_csv('csv/us_metros.csv')

db_name: str = 'us_metro_pops.db'
table_name: str = 'us_metro'

conn: Connection = sqlite3.connect(db_name)
cur: Cursor = conn.cursor()

df_to_sql(df, cur, table_name, db_name, conn) 

Table already exists.
